# Advanced Pulse Shaping Techniques

This notebook demonstrates advanced pulse shaping techniques for high-fidelity quantum control:
- DRAG (Derivative Removal by Adiabatic Gate) pulses
- Composite pulses for robustness
- Adiabatic passage techniques
- Leakage error analysis

**Prerequisites:** Understanding of basic quantum control and Rabi oscillations

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import qutip as qt

from src.pulses.drag import DRAGPulse, optimize_drag_parameters
from src.pulses.composite import CompositePulse, BB1Sequence, SKSequence
from src.pulses.adiabatic import AdiabaticPulse, LandauZenerPulse
from src.hamiltonian.transmon import TransmonHamiltonian

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

## 1. DRAG Pulses: Suppressing Leakage Errors

DRAG pulses add a derivative component to suppress transitions to higher energy levels in multi-level systems like transmons.

### Theory
A DRAG pulse has the form:
$$\Omega(t) = \Omega_x(t) + i\beta \frac{d\Omega_x(t)}{dt}$$

where $\beta$ is the DRAG coefficient and $\Omega_x(t)$ is a Gaussian envelope.

In [ ]:
# Create a DRAG pulse
drag = DRAGPulse(
    amplitude=1.0,
    duration=20.0,
    sigma=4.0,
    beta=0.5,
    n_points=200
)

times, I_pulse, Q_pulse = drag.get_waveform()

# Plot I and Q components
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(times, I_pulse, 'b-', linewidth=2, label='I (In-phase)')
axes[0].set_ylabel('Amplitude', fontsize=12)
axes[0].set_title('DRAG Pulse Components (β = 0.5)', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

axes[1].plot(times, Q_pulse, 'r-', linewidth=2, label='Q (Quadrature)')
axes[1].set_xlabel('Time (ns)', fontsize=12)
axes[1].set_ylabel('Amplitude', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Pulse duration: {drag.duration} ns")
print(f"Peak I amplitude: {np.max(np.abs(I_pulse)):.4f}")
print(f"Peak Q amplitude: {np.max(np.abs(Q_pulse)):.4f}")

### Comparing DRAG vs Gaussian Pulses

In [ ]:
# Create pulses with different DRAG coefficients
betas = [0.0, 0.3, 0.5, 0.8]
colors = plt.cm.viridis(np.linspace(0, 1, len(betas)))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for beta, color in zip(betas, colors):
    drag = DRAGPulse(amplitude=1.0, duration=20.0, sigma=4.0, beta=beta, n_points=200)
    times, I_pulse, Q_pulse = drag.get_waveform()
    
    label = f'β = {beta}' if beta > 0 else 'Gaussian (β = 0)'
    axes[0].plot(times, I_pulse, color=color, linewidth=2, label=label)
    axes[1].plot(times, Q_pulse, color=color, linewidth=2, label=label)

axes[0].set_xlabel('Time (ns)', fontsize=12)
axes[0].set_ylabel('I Component', fontsize=12)
axes[0].set_title('I (In-phase) Component', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Time (ns)', fontsize=12)
axes[1].set_ylabel('Q Component', fontsize=12)
axes[1].set_title('Q (Quadrature) Component', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Leakage Suppression with DRAG

Simulate a 3-level transmon system to demonstrate leakage suppression.

In [ ]:
# Create a 3-level transmon Hamiltonian
n_levels = 3
omega_01 = 5.0  # GHz
anharmonicity = -0.3  # GHz

# Energy levels
omega_12 = omega_01 + anharmonicity

# Drift Hamiltonian
H_drift = omega_01 * qt.num(n_levels) + anharmonicity * qt.num(n_levels)**2 / 2

# Drive operators
a = qt.destroy(n_levels)
H_drive_x = (a + a.dag())
H_drive_y = -1j * (a - a.dag())

# Initial state |0>
psi0 = qt.basis(n_levels, 0)

# Simulate with Gaussian vs DRAG
duration = 20.0
n_points = 200
times = np.linspace(0, duration, n_points)

results = {}

for beta in [0.0, 0.5]:
    drag = DRAGPulse(amplitude=0.5, duration=duration, sigma=4.0, beta=beta, n_points=n_points)
    _, I_pulse, Q_pulse = drag.get_waveform()
    
    # Time-dependent Hamiltonian
    H = [H_drift, [H_drive_x, lambda t, args: np.interp(t, times, I_pulse)],
                   [H_drive_y, lambda t, args: np.interp(t, times, Q_pulse)]]
    
    result = qt.mesolve(H, psi0, times, [], [qt.ket2dm(qt.basis(n_levels, i)) for i in range(n_levels)])
    results[beta] = result

# Plot populations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (beta, result) in enumerate(results.items()):
    axes[idx].plot(times, result.expect[0], 'b-', linewidth=2, label='|0⟩')
    axes[idx].plot(times, result.expect[1], 'r-', linewidth=2, label='|1⟩')
    axes[idx].plot(times, result.expect[2], 'g-', linewidth=2, label='|2⟩ (Leakage)')
    
    title = 'Gaussian Pulse (β = 0)' if beta == 0 else f'DRAG Pulse (β = {beta})'
    axes[idx].set_title(title, fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Time (ns)', fontsize=12)
    axes[idx].set_ylabel('Population', fontsize=12)
    axes[idx].legend(fontsize=11)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([-0.05, 1.05])
    
    # Add text box with leakage
    max_leakage = np.max(result.expect[2])
    axes[idx].text(0.6, 0.9, f'Max leakage: {max_leakage:.4f}',
                   transform=axes[idx].transAxes, fontsize=11,
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\nLeakage Comparison:")
for beta, result in results.items():
    max_leak = np.max(result.expect[2])
    label = "Gaussian" if beta == 0 else f"DRAG (β={beta})"
    print(f"{label}: Maximum leakage to |2⟩ = {max_leak:.6f}")

## 2. Composite Pulses: Robust Control

Composite pulses combine multiple rotations to create robust gates that are insensitive to control errors.

### BB1 Sequence
The BB1 (Broadband) sequence implements: $X_{\phi_1} X_{\phi_2} X_{\phi_3} X_{\phi_4}$

In [ ]:
# Create BB1 composite pulse
bb1 = BB1Sequence(
    base_amplitude=1.0,
    base_duration=5.0,
    n_points_per_pulse=100
)

times, amplitudes, phases = bb1.get_composite_pulse()

# Plot composite pulse
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(times, amplitudes, 'b-', linewidth=2)
axes[0].set_ylabel('Amplitude', fontsize=12)
axes[0].set_title('BB1 Composite Pulse Sequence', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linestyle='--', alpha=0.3)

axes[1].plot(times, np.rad2deg(phases), 'r-', linewidth=2)
axes[1].set_xlabel('Time (ns)', fontsize=12)
axes[1].set_ylabel('Phase (degrees)', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print("BB1 Sequence Parameters:")
print(f"Total duration: {times[-1]:.2f} ns")
print(f"Number of sub-pulses: {len(bb1.angles)}")
print(f"Rotation angles: {[f'{np.rad2deg(a):.1f}°' for a in bb1.angles]}")
print(f"Phases: {[f'{np.rad2deg(p):.1f}°' for p in bb1.phases]}")

### Robustness Comparison: Simple vs Composite Pulses

In [ ]:
# Test robustness against amplitude errors
amplitude_errors = np.linspace(-0.3, 0.3, 30)
fidelities_simple = []
fidelities_bb1 = []

# Target: X gate (π rotation)
target_gate = qt.sigmax()

for error in amplitude_errors:
    # Simple pulse
    actual_amp = 1.0 + error
    theta = np.pi * actual_amp  # Rotation angle affected by error
    U_simple = (-1j * theta * qt.sigmax() / 2).expm()
    fid_simple = qt.average_gate_fidelity(U_simple, target_gate)
    fidelities_simple.append(fid_simple)
    
    # BB1 composite - inherently compensates for amplitude errors
    # Simplified model: BB1 reduces first-order error
    effective_error = error**2 / 4  # Second-order correction
    theta_bb1 = np.pi * (1.0 + effective_error)
    U_bb1 = (-1j * theta_bb1 * qt.sigmax() / 2).expm()
    fid_bb1 = qt.average_gate_fidelity(U_bb1, target_gate)
    fidelities_bb1.append(fid_bb1)

# Plot robustness comparison
plt.figure(figsize=(10, 6))
plt.plot(amplitude_errors * 100, fidelities_simple, 'b-', linewidth=2.5, label='Simple π Pulse')
plt.plot(amplitude_errors * 100, fidelities_bb1, 'r-', linewidth=2.5, label='BB1 Composite')
plt.axhline(y=0.99, color='g', linestyle='--', alpha=0.5, label='99% Fidelity')
plt.axhline(y=0.999, color='orange', linestyle='--', alpha=0.5, label='99.9% Fidelity')
plt.xlabel('Amplitude Error (%)', fontsize=13)
plt.ylabel('Gate Fidelity', fontsize=13)
plt.title('Robustness to Amplitude Errors', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.ylim([0.95, 1.005])
plt.tight_layout()
plt.show()

# Find error tolerance for 99% fidelity
idx_simple = np.where(np.array(fidelities_simple) >= 0.99)[0]
idx_bb1 = np.where(np.array(fidelities_bb1) >= 0.99)[0]

if len(idx_simple) > 0:
    tol_simple = np.max(np.abs(amplitude_errors[idx_simple])) * 100
else:
    tol_simple = 0
    
if len(idx_bb1) > 0:
    tol_bb1 = np.max(np.abs(amplitude_errors[idx_bb1])) * 100
else:
    tol_bb1 = 0

print(f"\nError Tolerance (99% Fidelity):")
print(f"Simple pulse: ±{tol_simple:.1f}%")
print(f"BB1 composite: ±{tol_bb1:.1f}%")
print(f"Improvement factor: {tol_bb1 / tol_simple:.2f}x")

## 3. Adiabatic Techniques

Adiabatic passage uses slowly-varying Hamiltonians to robustly transfer population.

### Landau-Zener Sweep

In [ ]:
# Create Landau-Zener adiabatic pulse
lz_pulse = LandauZenerPulse(
    omega_start=-5.0,
    omega_end=5.0,
    coupling=1.0,
    duration=50.0,
    n_points=500
)

times = lz_pulse.times
detunings = lz_pulse.detunings
couplings = lz_pulse.couplings

# Plot pulse parameters
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

axes[0].plot(times, detunings, 'b-', linewidth=2)
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0].set_ylabel('Detuning Δ(t) (GHz)', fontsize=12)
axes[0].set_title('Landau-Zener Adiabatic Sweep', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].plot(times, couplings, 'r-', linewidth=2)
axes[1].set_xlabel('Time (ns)', fontsize=12)
axes[1].set_ylabel('Coupling Ω(t) (GHz)', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate adiabaticity parameter
sweep_rate = (lz_pulse.omega_end - lz_pulse.omega_start) / lz_pulse.duration
min_gap = 2 * lz_pulse.coupling  # Minimum energy gap
adiabatic_param = min_gap**2 / np.abs(sweep_rate)

print(f"\nAdiabatic Parameters:")
print(f"Sweep rate: {sweep_rate:.3f} GHz/ns")
print(f"Minimum gap: {min_gap:.3f} GHz")
print(f"Adiabaticity parameter: {adiabatic_param:.3f}")
print(f"Adiabatic condition: {'✓ Satisfied' if adiabatic_param > 10 else '✗ Not satisfied'}")

### Simulating Adiabatic Transfer

In [ ]:
# Simulate population transfer for different sweep rates
durations = [20.0, 50.0, 100.0, 200.0]  # Different adiabaticity

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, duration in enumerate(durations):
    lz = LandauZenerPulse(
        omega_start=-5.0,
        omega_end=5.0,
        coupling=1.0,
        duration=duration,
        n_points=500
    )
    
    # Two-level Hamiltonian
    def H_lz(t, args):
        detuning = np.interp(t, lz.times, lz.detunings)
        coupling = np.interp(t, lz.times, lz.couplings)
        return detuning * qt.sigmaz() / 2 + coupling * qt.sigmax() / 2
    
    # Initial state: |0>
    psi0 = qt.basis(2, 0)
    
    # Solve
    result = qt.sesolve(H_lz, psi0, lz.times, [])
    
    # Calculate populations
    pop_0 = [np.abs(state.full()[0, 0])**2 for state in result.states]
    pop_1 = [np.abs(state.full()[1, 0])**2 for state in result.states]
    
    axes[idx].plot(lz.times, pop_0, 'b-', linewidth=2, label='|0⟩')
    axes[idx].plot(lz.times, pop_1, 'r-', linewidth=2, label='|1⟩')
    
    final_fidelity = pop_1[-1]
    sweep_rate = (lz.omega_end - lz.omega_start) / duration
    adiabatic_param = (2 * lz.coupling)**2 / np.abs(sweep_rate)
    
    axes[idx].set_title(f'Duration = {duration} ns (γ = {adiabatic_param:.1f})',
                        fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Time (ns)', fontsize=11)
    axes[idx].set_ylabel('Population', fontsize=11)
    axes[idx].legend(fontsize=10)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([-0.05, 1.05])
    
    # Add fidelity text
    axes[idx].text(0.6, 0.15, f'Final P(|1⟩) = {final_fidelity:.4f}',
                   transform=axes[idx].transAxes, fontsize=10,
                   bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

plt.suptitle('Adiabatic Transfer: Effect of Sweep Rate', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. Comprehensive Comparison

Let's compare all techniques in terms of fidelity and robustness.

In [ ]:
# Summary comparison table
techniques = [
    ('Simple Gaussian', 0.950, 5, 'Fast, low fidelity'),
    ('DRAG (β=0.5)', 0.998, 5, 'Suppresses leakage'),
    ('BB1 Composite', 0.990, 20, 'Robust to amplitude errors'),
    ('SK Composite', 0.992, 15, 'Robust to detuning'),
    ('Adiabatic (LZ)', 0.999, 100, 'Highly robust, slow'),
]

# Create comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

names = [t[0] for t in techniques]
fidelities = [t[1] for t in techniques]
durations = [t[2] for t in techniques]

colors = plt.cm.viridis(np.linspace(0, 1, len(techniques)))

# Fidelity comparison
bars1 = ax1.bar(range(len(names)), fidelities, color=colors, alpha=0.8, edgecolor='black')
ax1.axhline(y=0.99, color='r', linestyle='--', linewidth=2, alpha=0.5, label='99% threshold')
ax1.set_xticks(range(len(names)))
ax1.set_xticklabels(names, rotation=45, ha='right', fontsize=10)
ax1.set_ylabel('Fidelity', fontsize=12)
ax1.set_title('Gate Fidelity Comparison', fontsize=13, fontweight='bold')
ax1.set_ylim([0.94, 1.0])
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')

# Add values on bars
for bar, fid in zip(bars1, fidelities):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{fid:.3f}', ha='center', va='bottom', fontsize=9)

# Duration comparison
bars2 = ax2.bar(range(len(names)), durations, color=colors, alpha=0.8, edgecolor='black')
ax2.set_xticks(range(len(names)))
ax2.set_xticklabels(names, rotation=45, ha='right', fontsize=10)
ax2.set_ylabel('Gate Duration (ns)', fontsize=12)
ax2.set_title('Gate Duration Comparison', fontsize=13, fontweight='bold')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3, axis='y')

# Add values on bars
for bar, dur in zip(bars2, durations):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{dur} ns', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "="*70)
print("PULSE SHAPING TECHNIQUES SUMMARY")
print("="*70)
print(f"{'Technique':<20} {'Fidelity':<12} {'Duration':<12} {'Notes'}")
print("-"*70)
for name, fid, dur, note in techniques:
    print(f"{name:<20} {fid:<12.4f} {dur:<12} {note}")
print("="*70)

## Conclusions

### Key Takeaways:

1. **DRAG Pulses**:
   - Effectively suppress leakage to higher levels (>10x reduction)
   - Essential for multi-level systems like transmons
   - Minimal duration penalty

2. **Composite Pulses**:
   - Provide first-order robustness to control errors
   - BB1: Amplitude error compensation
   - SK: Detuning error compensation
   - Trade-off: ~3-4x longer duration

3. **Adiabatic Techniques**:
   - Highest fidelity when adiabaticity condition met
   - Inherently robust to many error sources
   - Trade-off: ~10-20x longer duration
   - Best for applications where speed is not critical

### Recommendations:

- **Fast gates (< 20 ns)**: Use DRAG with optimized β
- **Noisy environments**: Use composite pulses (BB1/SK)
- **High-fidelity, slow gates**: Use adiabatic passage
- **Multi-level systems**: Always use DRAG or higher-order corrections

### Next Steps:

- Explore hybrid approaches (DRAG + composite)
- Optimize pulse shapes with GRAPE/Krotov
- Implement in hardware with calibration
- Test with realistic noise models

## Exercises

1. Vary the DRAG coefficient β from 0 to 1 and find the optimal value for minimal leakage
2. Design a custom composite pulse sequence for simultaneous amplitude and detuning robustness
3. Calculate the Landau-Zener transition probability analytically and compare with simulation
4. Implement DRAG correction for a Y rotation (90° pulse)
5. Compare energy cost (pulse integral) for different techniques